<a href="https://colab.research.google.com/github/vijayakanth/Transfer-Learning/blob/main/teaLeave_InsceptionV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


# !pip install split-folders



In [2]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import ImageDataGenerator,load_img
from keras.layers import Activation, Dropout, Flatten, Dense,Input
from sklearn.model_selection  import train_test_split,KFold
from tensorflow.keras import regularizers
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import utils as np_utils
# from tensorflow.keras.applications.vgg16 import VGG16
from matplotlib import pyplot
from keras.models import Sequential
import tensorflow as tf
from keras import backend as K
from tensorflow.keras.optimizers import Adam
import tensorflow.keras
import numpy as np
import glob
import cv2
import os

In [3]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [1]:
# def getImages(no_of_classes):
#     disease_images = []
#     labels = []
#     i = 0
#     for disease_image_path in glob.glob("/content/drive/MyDrive/ColabNotebooks/data/teaLeaveDisease/*"):
#         disease_label = disease_image_path.split("/")[-1]
#         if no_of_classes == i:
#             break
#         for image_path in glob.glob(os.path.join(disease_image_path, "*.jpg")):
#             # image = cv2.imread(image_path, cv2.IMREAD_COLOR)
#             image = cv2.imread(image_path)
#             image = cv2.resize(image, (224, 224),interpolation = cv2.INTER_CUBIC)
#             # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

#             disease_images.append(image)
#             labels.append(disease_label)
#         i = i + 1

#     disease_images = np.array(disease_images)
#     labels = np.array(labels)
#     label_to_id_dict = {v: i for i, v in enumerate(np.unique(labels))}
#     id_to_label_dict = {v: k for k, v in label_to_id_dict.items()}

#     label_ids = np.array([label_to_id_dict[x] for x in labels])
#     disease_images.shape, label_ids.shape, labels.shape

#     print ("Total Images : " +  str(disease_images.shape[0]))
#     # print ("Total Labels : " +  str(label_ids))
#     return disease_images, label_ids

In [ ]:
# disease, labels = getImages(6)


In [ ]:
# # dimensions of our images.
Image_size = [299,299]
# img_width, img_height=224,224
batchSize=32

In [ ]:
#augmentation ,we will use for training and not in validation and testing
train_datagen= ImageDataGenerator(rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale=1. / 255)
test_datagen = ImageDataGenerator(rescale=1. / 255)


In [ ]:
# import splitfolders
# import os
# path = "/content/drive/MyDrive/ColabNotebooks/data/teaLeaveDisease/"
# print(os.listdir(path))

['Blister_blight', 'Spider_mites', 'Black_blight', 'Gray_blight', 'Algal_leaf_spot', 'Healthy_leaf']


In [ ]:
# splitfolders.ratio(path,seed=1337, output="/content/drive/MyDrive/ColabNotebooks/data/disease-Splitted", ratio=(0.7, 0.1, 0.2))

Copying files: 1104 files [00:19, 55.35 files/s]


In [ ]:
train="/content/drive/MyDrive/ColabNotebooks/data/disease-Splitted/train"
val="/content/drive/MyDrive/ColabNotebooks/data/disease-Splitted/val"
test="/content/drive/MyDrive/ColabNotebooks/data/disease-Splitted/test"
#Training dataset
train_set = train_datagen.flow_from_directory(
                train,
                 target_size=Image_size,
                batch_size=batchSize,
                interpolation='bicubic',
                class_mode='categorical',
                shuffle=True)
#Validation dataset
valid_set= valid_datagen.flow_from_directory(
              val,
              target_size=Image_size,
              batch_size=batchSize,
              interpolation='bicubic',
              class_mode='categorical',
              shuffle=True)
#Testing dataset
test_set= test_datagen.flow_from_directory(
              test,
              target_size=Image_size,
              batch_size=1,
              interpolation='bicubic',
              class_mode='categorical',
              shuffle=False)

Found 770 images belonging to 6 classes.
Found 107 images belonging to 6 classes.
Found 227 images belonging to 6 classes.


In [ ]:
# Load pre-trained InceptionV3 model (excluding top layers)
inception = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# # Freeze layers except the top ones for fine-tuning
# for layer in inception.layers[:-10]:
#     layer.trainable = False

In [ ]:
#don't train the existing weights
for layer in inception.layers:
  layer.trainable=False
# I chose to train the top 2 inception blocks, i.e. I will freeze the first 249 layers and unfreeze the rest:
for layer in inception.layers[:249]:
   layer.trainable = False
for layer in inception.layers[249:]:
   layer.trainable = True

x=Flatten()(inception.output)
x=Dense(512,activation='relu',kernel_regularizer=regularizers.l1(0.001))(x)
x=Dense(256,activation='relu',kernel_regularizer=regularizers.l1(0.001))(x)
x=Dropout(rate=0.3)(x)
prediction=Dense(6,activation='softmax')(x)
model=Model(inputs=inception.input,outputs=prediction)

In [ ]:
# model.summary()

In [ ]:
 # Adam optinization
opt =Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])

In [ ]:
# from tensorflow.keras.optimizers import SGD
# model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history=model.fit(
                  train_set,
                  steps_per_epoch=train_set.samples // batchSize,
                  epochs=100,
                  validation_data=valid_set,
                  validation_steps=valid_set.samples // batchSize, callbacks=[tf.keras.callbacks.EarlyStopping
	                            (monitor='val_loss',patience=10)], verbose=2 )




Epoch 1/50
24/24 - 191s - loss: 4.5596 - accuracy: 0.9878 - val_loss: 4.6523 - val_accuracy: 0.9375 - 191s/epoch - 8s/step
Epoch 2/50
24/24 - 171s - loss: 4.6145 - accuracy: 0.9756 - val_loss: 6.1059 - val_accuracy: 0.9167 - 171s/epoch - 7s/step
Epoch 3/50
24/24 - 172s - loss: 5.3314 - accuracy: 0.9824 - val_loss: 4.7288 - val_accuracy: 0.9271 - 172s/epoch - 7s/step
Epoch 4/50
24/24 - 191s - loss: 4.1982 - accuracy: 0.9946 - val_loss: 4.1812 - val_accuracy: 0.9167 - 191s/epoch - 8s/step
Epoch 5/50
24/24 - 174s - loss: 4.4175 - accuracy: 0.9756 - val_loss: 6.0360 - val_accuracy: 0.9583 - 174s/epoch - 7s/step
Epoch 6/50
24/24 - 188s - loss: 5.2280 - accuracy: 0.9878 - val_loss: 4.8174 - val_accuracy: 0.8958 - 188s/epoch - 8s/step
Epoch 7/50
24/24 - 172s - loss: 4.1299 - accuracy: 0.9851 - val_loss: 5.6550 - val_accuracy: 0.8750 - 172s/epoch - 7s/step
Epoch 8/50
24/24 - 181s - loss: 6.5705 - accuracy: 0.9607 - val_loss: 7.8995 - val_accuracy: 0.9271 - 181s/epoch - 8s/step
Epoch 9/50
24/24

In [ ]:
model.evaluate(test_set,steps=len(test_set))

227/227 [==============================] - 45s 187ms/step - loss: 5.8336 - accuracy: 0.9515


[5.833595275878906, 0.9515418410301208]

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
# Evaluate the model on test data
true_labels = test_set.classes
predicted_labels = model.predict(test_set)
predicted_labels = tf.argmax(predicted_labels, axis=1).numpy()

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy: {accuracy:.4f}")

# Generate classification report including precision, recall, and F1-score for each class
report = classification_report(true_labels, predicted_labels)
print("Classification Report:\n", report)

In [ ]:
# new_model=tf.keras.models.load_model('/content/drive/MyDrive/ColabNotebooks_tranferLearning/first_try_01_Model_InsceptionV3.h5')
# new_model.evaluate(test_set)

In [ ]:
model.save_weights('/content/drive/MyDrive/ColabNotebooks/512_256_weights_InsceptionV3.h5')
model.save('/content/drive/MyDrive/ColabNotebooks/512_256_InsceptionV3.h5')

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.densenet import preprocess_input

# Load DenseNet-201 model (pre-trained on ImageNet)
model_path ='/content/drive/MyDrive/ColabNotebooks/512_256_InsceptionV3.h5'
load_model = tf.keras.models.load_model(model_path)

# Choose an image path for which you want to generate the heatmap
image_path = '/content/drive/MyDrive/Spider_mites.jpg'  # Replace with the path to your image

# Load and preprocess the image
img = image.load_img(image_path, target_size=(229,229))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

# Get the predictions for the image
preds = load_model.predict(x)
predicted_class = np.argmax(preds[0])
print("Predicted class:", predicted_class)

# Get the output of the last convolutional layer in the model
last_conv_layer = load_model.get_layer('conv5_block32_concat')  # Choose the appropriate layer from your model

# Create a model that maps the input image to the activations of the last conv layer
heatmap_model = tf.keras.models.Model([load_model.inputs], [last_conv_layer.output, load_model.output])

# Compute the gradients of the top predicted class with respect to the output feature map
with tf.GradientTape() as tape:
    conv_outputs, predictions = heatmap_model(x)
    loss = predictions[:, predicted_class]

grads = tape.gradient(loss, conv_outputs)
pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

# Multiply each channel in the feature map array by "how important this channel is" regarding the class
heatmap = tf.reduce_mean(tf.multiply(conv_outputs, pooled_grads[..., tf.newaxis]), axis=-1)
heatmap = np.maximum(heatmap, 0)  # ReLU to eliminate negative values
heatmap /= np.max(heatmap)  # Normalize values to range [0, 1]

# Resize heatmap to the original image size
heatmap = tf.image.resize(heatmap, (img.size[1], img.size[0]))

# Plot the original image with the heatmap overlay
plt.figure(figsize=(8, 6))
plt.imshow(img)
plt.imshow(heatmap[0], alpha=0.5, cmap='jet')
plt.colorbar()
plt.show()

OSError: No file or directory found at /content/drive/MyDrive/ColabNotebooks/512_256_InsceptionV3.keras

In [ ]:
def loss(history):
  #plot loss
  pyplot.subplot(211)
  pyplot.title('Cross Entropy Loss')
  pyplot.xlabel('Epochs')
  pyplot.ylabel('Loss')
  pyplot.plot(history.history['loss'], color='blue', label='training')
  pyplot.plot(history.history['val_loss'], color='orange', label='validation')
  pyplot.legend()
def accuracy(history):
  #plot accuracy
  pyplot.subplot(212)
  pyplot.title('Classification Accuracy')
  pyplot.xlabel('Epochs')
  pyplot.ylabel('accuracy')
  pyplot.plot(history.history['accuracy'], color='blue', label='training')
  pyplot.plot(history.history['val_accuracy'], color='orange', label='validation')
  pyplot.legend()

In [ ]:
loss(history)

In [ ]:
accuracy(history)